## 文本分类应用一: 有监督长文本分类


### 数据
使用THUNews! THUCNews是根据新浪新闻RSS订阅频道2005~2011年间的历史数据筛选过滤生成，包含74万篇新闻文档（2.19 GB），均为UTF-8纯文本格式。在原始新浪新闻分类体系的基础上，重新整合划分出14个候选分类类别：财经、彩票、房产、股票、家居、教育、科技、社会、时尚、时政、体育、星座、游戏、娱乐。

本实验截取其中约30万数据进行训练

具体介绍可见 [链接](http://thuctc.thunlp.org/#%E4%B8%AD%E6%96%87%E6%96%87%E6%9C%AC%E5%88%86%E7%B1%BB%E6%95%B0%E6%8D%AE%E9%9B%86THUCNews)


#### 下载数据

In [0]:
!curl -O http://thunlp.oss-cn-qingdao.aliyuncs.com/THUCNews.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1488M  100 1488M    0     0  2427k      0  0:10:27  0:10:27 --:--:-- 2340k32k:10  0:08:43 2386k 4925k2 2578k:21  0:07:50  0:02:31 2583k  0  0:10:26  0:09:59  0:00:27 2208k2325k


#### 解压文件
可能的问题: 解压后文本内容都是正常的,但是文件夹名字会是乱码

In [0]:
import zipfile
with zipfile.ZipFile("THUCNews.zip", 'r') as zip_ref:
    zip_ref.extractall()

In [0]:
ls

fastText-0.9.1/          v0.9.1.zip  娱乐/  房产/  星座/  科技/
news_fasttext_test.txt   体育/       家居/  教育/  游戏/  股票/
news_fasttext_train.txt  军事/       彩票/  时尚/  社会/  财经/


In [0]:
# pip install jieba
# pip install fasttext

Note: you may need to restart the kernel to use updated packages.


In [0]:
base_dir = "/home/real3stone/data/THUCNews/"

In [0]:
import os
os.chdir(base_dir)

### 处理数据
因为fastText要求其数的label必须在首/尾以__label__形式给出,所以需要简单处理一下

**训练集和验证集 组成:** 
截取每一个类别中的前15000条为训练集,之后5000条为验证集;

In [0]:
import jieba
import pandas 
import fastext
import os

dir_list = ['体育','军事','娱乐','家居','彩票','房产','教育','时尚','星座','游戏','社会','科技','股票', '财经']
##生成fastext的训练和验证数据集

ftrain = open("news_fasttext_train.txt","w")
ftest = open("news_fasttext_test.txt","w")

num = -1
for e in dir_list:
    num += 1
    indir = base_dir + e + '/'
    files = os.listdir(indir)
    count = 0
    for fileName in files:
        count += 1            
        filepath = indir + fileName
        with open(filepath,'r') as fr:
            text = fr.read()
#         text = text.decode("utf-8").encode("utf-8") # ERROR: 'str' object has no attribute 'decode'
        seg_text = jieba.cut(text.replace("\t"," ").replace("\n"," "))
        outline = " ".join(seg_text)
#         outline = outline.encode("utf-8") + "\t__label__" + e + "\n"
        outline = outline + "\t__label__" + e + "\n"

        //截取验证集
        if count < 15000:
            ftrain.write(outline)
            ftrain.flush()
            continue
        //截取验证机
        elif count  < 20000:
            ftest.write(outline)
            ftest.flush()
            continue
        else:
            break

ftrain.close()
ftest.close()

In [0]:
ls

fastText-0.9.1/          v0.9.1.zip  娱乐/  房产/  星座/  科技/
news_fasttext_test.txt   体育/       家居/  教育/  游戏/  股票/
news_fasttext_train.txt  军事/       彩票/  时尚/  社会/  财经/


### 训练模型

#### loss使用hierarchical_softmax
学习率初始值设置为1.0(因为会逐渐衰减,一开始大一点不会是效果变差,但可以加快训练速度,), epoch设置为25

In [0]:
classifier = fasttext.train_supervised("news_fasttext_train.txt", lr=1.0, epoch=25, loss='hs')

测试训练效果:

In [0]:
classifier.test("news_fasttext_test.txt")

(113369, 0.9057149661724104, 0.9057149661724104)

#### 改进
loss采用one-vs-all方法(转为多个二分类), 并引入n-grams(选择2-grams), 并利用hash桶优化; 词向量维度选择80;

In [0]:
classifier = fasttext.train_supervised("news_fasttext_train.txt", lr=1.0, epoch=25, loss='ova', wordNgrams=2, bucket=200000, dim=80)

查看训练效果

In [0]:
classifier.test("news_fasttext_test.txt")

(113369, 0.9308364720514426, 0.9308364720514426)

## 文本分类应用二: 无监督学习

实验内容: 对头条新闻的标题(短文本)进行无监督学习, 生成词向量

数据来源: [链接](https://github.com/fatecbf/toutiao-text-classfication-dataset)

数据规模: 共382688条，分布于15个分类中。

In [0]:
base_dir = "/home/real3stone/data"

In [0]:
import pandas as pd
import fasttext
import os
import jieba
os.chdir(base_dir)

### 数据处理

- **原始数据格式:** 6552431613437805063_!_102_!_news_entertainment_!_谢娜为李浩菲澄清网络谣言，之后她的两个行为给自己加分t_!_佟丽娅,网络谣言,快乐大本营,李浩菲,谢娜,观众们

- **原始数据含义:** 每行为一条数据，以_!_分割字段，依次表示新闻ID，分类code，分类名称，新闻字符串，新闻关键词


使用pandas读取,并处理为fastText可以使用的格式

In [0]:
ls

THUCNews/               news_fasttext_train.txt      v0.9.1.zip
THUCNews.zip            stopwords-master/            中文停用词表.txt
__MACOSX/               toutiao_cat_data.txt
news_fasttext_test.txt  unsupervised_train_data.txt


In [0]:
news_data = pd.read_csv('toutiao_cat_data.txt', sep="_!_", header=None)
news_data.columns = ["id", "code", "label", "content", "key_words"]
news_data.dropna() # 删去空行

/home/real3stone/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


,id,code,label,content,key_words
0,6551700932705387022,101,news_culture,京城最值得你来场文化之旅的博物馆,"保利集团,马未都,中国科学技术馆,博物馆,新中国"
10,6552387367334838792,101,news_culture,国画山水，如何读懂山水画,"林风眠,黄海归来步步云,秋山图,计白当黑,山水画,江山万里图,张大千,巫峡清秋图,活眼,山雨欲来图"
11,6552314684383429128,101,news_culture,一元硬币是这种，现在价值24000元，找找看！,"牡丹,收藏价值"
14,6552301380562846215,101,news_culture,赵宁安作品欣赏,"叶浅予,田世光,李苦禅,花鸟画,中央美术学院"
19,6552466638304707079,101,news_culture,初夏方好，小荷初露，一起来读那些美不胜收的咏荷经典诗词,"荷花,西湖,金粟词话,采莲女,念奴娇·赤壁怀古,林逋,荷叶"
...,...,...,...,...,...
382678,6554596645437178371,104,news_finance,百万亏损中悟出的交易之道,"齐威王,田忌赛马,忌讳,同龄人,交易者,好朋友,做交易,股民,股票市场,田忌"
382679,6554627652047602190,107,news_car,这款合资车曾比朗逸还火 现在却成功“跳楼” 售价仅为8万！,"科鲁兹,小轿车,雪佛兰,大众朗逸,SUV"
382681,6554661690015744516,107,news_car,精致实用，这辆房车专为行家准备,"车内,C200,房车,水曲柳,依维柯,玻璃钢"
382685,6554489948580348424,113,news_world,又一国领导人放话，只要普京下令，数万大军“碾压”美国白宫！,"以色列,普京,俄罗斯,叙利亚,车臣"


In [0]:
news_contents = news_data.content.values.tolist()
news_label = news_data.label.values.tolist()

In [0]:
news_contents[:5], news_label[:5]

(['京城最值得你来场文化之旅的博物馆',
  '发酵床的垫料种类有哪些？哪种更好？',
  '上联：黄山黄河黄皮肤黄土高原。怎么对下联？',
  '林徽因什么理由拒绝了徐志摩而选择梁思成为终身伴侣？',
  '黄杨木是什么树？'],
 ['news_culture',
  'news_culture',
  'news_culture',
  'news_culture',
  'news_culture'])

#### 去停用词
使用中文停用词表.txt,[Github链接](https://github.com/goto456/stopwords)

In [0]:
def getStopWords(datapath):
    stopwords=pd.read_csv(datapath,index_col=False,quoting=3,sep="\t",names=['stopword'], encoding='utf-8')
    stopwords=stopwords["stopword"].values
    return stopwords

In [0]:
def preprocess_text(content_line,sentences,stopwords):
    for line in content_line:
        try:
            segs=jieba.lcut(line)    #利用结巴分词进行中文分词
            segs=filter(lambda x:len(x)>1,segs)    #去掉长度小于1的词
            segs=filter(lambda x:x not in stopwords,segs)    #去掉停用词
            sentences.append(" ".join(segs))
        except Exception as e:
            print (line)
            continue

In [0]:
stopwordsFile=r"./中文停用词表.txt"
stopwords=getStopWords(stopwordsFile)

In [0]:
toutiao_for_unsupervised = r'unsupervised_train_data.txt'
news_for_fasttext = []
preprocess_text(news_contents, news_for_fasttext, stopwords)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.792 seconds.
Prefix dict has been built succesfully.


In [0]:
news_for_fasttext[:5]

['京城 值得 来场 文化 之旅 博物馆',
 '发酵 垫料 种类 更好',
 '上联 黄山 黄河 黄皮肤 黄土高原 下联',
 '林徽因 理由 拒绝 徐志摩 选择 梁思成 终身伴侣',
 '黄杨木']

In [0]:
# 写入文件
def writeData(contents,fileName):
    print("writing data to fasttext format...")
    out=open(fileName,'w')
    for content_line in contents:
        out.write(content_line+"\n")
    print("write done!")

In [0]:
import random
random.shuffle(news_for_fasttext)    #做乱序处理，使得同类别的样本不至于扎堆
writeData(news_for_fasttext,toutiao_for_unsupervised)

writing data to fasttext format...
write done!


In [0]:
model_skipgram=fasttext.train_unsupervised('unsupervised_train_data.txt',model='skipgram')

In [0]:
print(model_skipgram.words[:100])    #打印词向量

['</s>', '中国', '美国', '上联', '下联', '网友', '一个', '手机', '2018', '世界', '现在', '没有', '农村', '王者', '荣耀', '10', '日本', '看待', '知道', '游戏', '到底', '真的', '国家', '活动', '英雄', '联想', '汽车', '俄罗斯', '特朗普', '孩子', '20', '小米', '房价', '旅游', '成为', '评价', '第一', '未来', '火箭', '伊朗', '以色列', '城市', '房子', '协议', '老师', '买房', '选择', '詹姆斯', '需要', '是否', '这种', '不能', '发展', '学生', '华为', '喜欢', '应该', 'NBA', '公司', '不会', '投资', '出席', '企业', '值得', '勇士', '微信', '可能', '马云', 'SUV', '大学', '万元', '上市', '推荐', '联盟', '印度', '市场', '总统', '比赛', '骑士', '全球', '影响', '绝地', '北京', '求生', '退出', '最后', '问题', '技术', '比较', '苹果', '地方', '已经', '原因', '教育', '认为', '即将', '玩家', '生活', '价格', '腾讯']


In [0]:
model_cbow=fasttext.train_unsupervised('unsupervised_train_data.txt',model='cbow')

In [0]:
print(model_cbow.words[:100])    #打印词向量

['</s>', '中国', '美国', '上联', '下联', '网友', '一个', '手机', '2018', '世界', '现在', '没有', '农村', '王者', '荣耀', '10', '日本', '看待', '知道', '游戏', '到底', '真的', '国家', '活动', '英雄', '联想', '汽车', '俄罗斯', '特朗普', '孩子', '20', '小米', '房价', '旅游', '成为', '评价', '第一', '未来', '火箭', '伊朗', '以色列', '城市', '房子', '协议', '老师', '买房', '选择', '詹姆斯', '需要', '是否', '这种', '不能', '发展', '学生', '华为', '喜欢', '应该', 'NBA', '公司', '不会', '投资', '出席', '企业', '值得', '勇士', '微信', '可能', '马云', 'SUV', '大学', '万元', '上市', '推荐', '联盟', '印度', '市场', '总统', '比赛', '骑士', '全球', '影响', '绝地', '北京', '求生', '退出', '最后', '问题', '技术', '比较', '苹果', '地方', '已经', '原因', '教育', '认为', '即将', '玩家', '生活', '价格', '腾讯']
